In [1]:
from abstractions.goap.spatial import GridMap, GameEntity, BlocksMovement, BlocksLight
from pydantic import ValidationError

# Create a grid map
grid_map = GridMap(width=10, height=10)

# Create some game entities
player = GameEntity(name="Player")
wall1 = GameEntity(name="Wall1", blocks_movement=BlocksMovement(value=True))
wall2 = GameEntity(name="Wall2", blocks_movement=BlocksMovement(value=True))
goal = GameEntity(name="Goal")

# Place entities on the grid map
grid_map.get_node((1, 1)).add_entity(player)
grid_map.get_node((3, 3)).add_entity(wall1)
grid_map.get_node((5, 5)).add_entity(wall2)
grid_map.get_node((8, 8)).add_entity(goal)

# Find the shortest path from the player's position to the goal using A*
start_node = grid_map.get_node(player.position.value)
goal_node = grid_map.get_node(goal.position.value)
path = grid_map.a_star(start_node, goal_node)

# Print the grid map with the path
print("Grid Map with A* Path:")
grid_map.print_grid(path)

# Find distances and paths from the player's position to all reachable nodes using Dijkstra
max_distance = 10
distances, paths = grid_map.dijkstra(start_node, max_distance)

# Print the distances and paths
print("\nDistances (Dijkstra):")
for node, distance in distances.items():
    print(f"Node at position {node}: Distance = {distance}")

print("\nPaths (Dijkstra):")
for node, path in paths.items():
    print(f"Node at position {node}: Path = {[n.position.value for n in path.nodes]}")

Grid Map with A* Path:
. . . . . . . . . . 
. S . . . . . . . . 
. . * . . . . . . . 
. . * # . . . . . . 
. . . * . . . . . . 
. . . . * # . . . . 
. . . . . * . . . . 
. . . . . . * . . . 
. . . . . . . * G . 
. . . . . . . . . . 

Distances (Dijkstra):
Node at position (1, 1): Distance = 0
Node at position (1, 2): Distance = 1
Node at position (1, 0): Distance = 1
Node at position (2, 1): Distance = 1
Node at position (0, 1): Distance = 1
Node at position (2, 2): Distance = 1
Node at position (0, 2): Distance = 1
Node at position (2, 0): Distance = 1
Node at position (0, 0): Distance = 1
Node at position (0, 3): Distance = 2
Node at position (1, 3): Distance = 2
Node at position (2, 3): Distance = 2
Node at position (3, 0): Distance = 2
Node at position (3, 1): Distance = 2
Node at position (3, 2): Distance = 2
Node at position (0, 4): Distance = 3
Node at position (1, 4): Distance = 3
Node at position (2, 4): Distance = 3
Node at position (3, 4): Distance = 3
Node at position (4, 0

In [2]:
from abstractions.goap.spatial import GridMap, GameEntity, BlocksMovement, BlocksLight
import random

def create_room(grid_map, top_left, width, height):
    for x in range(top_left[0], top_left[0] + width):
        for y in range(top_left[1], top_left[1] + height):
            grid_map.get_node((x, y)).entities.clear()

def create_h_corridor(grid_map, x1, x2, y):
    for x in range(min(x1, x2), max(x1, x2) + 1):
        grid_map.get_node((x, y)).entities.clear()

def create_v_corridor(grid_map, y1, y2, x):
    for y in range(min(y1, y2), max(y1, y2) + 1):
        grid_map.get_node((x, y)).entities.clear()

def generate_dungeon(grid_map, num_rooms, min_room_size, max_room_size):
    rooms = []
    for _ in range(num_rooms):
        width = random.randint(min_room_size, max_room_size)
        height = random.randint(min_room_size, max_room_size)
        x = random.randint(1, grid_map.width - width - 1)
        y = random.randint(1, grid_map.height - height - 1)
        create_room(grid_map, (x, y), width, height)
        rooms.append((x, y, width, height))

    for i in range(len(rooms) - 1):
        x1, y1, w1, h1 = rooms[i]
        x2, y2, w2, h2 = rooms[i + 1]

        if random.random() < 0.5:
            create_h_corridor(grid_map, x1 + w1, x2, y1 + h1 // 2)
            create_v_corridor(grid_map, y1 + h1 // 2, y2 + h2 // 2, x2)
        else:
            create_v_corridor(grid_map, y1 + h1 // 2, y2, x1 + w1 // 2)
            create_h_corridor(grid_map, x1 + w1 // 2, x2 + w2 // 2, y2)

# Create a larger grid map
grid_map = GridMap(width=50, height=50)

# Fill the grid map with walls
for x in range(grid_map.width):
    for y in range(grid_map.height):
        wall = GameEntity(name=f"Wall_{x}_{y}", blocks_movement=BlocksMovement(value=True))
        grid_map.get_node((x, y)).add_entity(wall)

# Generate the dungeon
num_rooms = 10
min_room_size = 5
max_room_size = 10
generate_dungeon(grid_map, num_rooms, min_room_size, max_room_size)

# Find valid floor tiles for player and goal
floor_tiles = [(x, y) for x in range(grid_map.width) for y in range(grid_map.height) if not grid_map.get_node((x, y)).blocks_movement]
player_pos, goal_pos = random.sample(floor_tiles, 2)

# Place the player and goal entities
player = GameEntity(name="Player")
goal = GameEntity(name="Goal")
grid_map.get_node(player_pos).add_entity(player)
grid_map.get_node(goal_pos).add_entity(goal)

# Find the shortest path from the player's position to the goal using A*
start_node = grid_map.get_node(player.position.value)
goal_node = grid_map.get_node(goal.position.value)
path = grid_map.a_star(start_node, goal_node)
shadow = grid_map.get_shadow(start_node, max_radius=10)

# Print the grid map with the path
print("Grid Map with A* Path:")
grid_map.print_grid(path)


Grid Map with A* Path:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # . . . . . . . . . . # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # . . . . . . . . . . # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # . . . . . . . . . . # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # . . . . . . . . . . . . . # # # # # # # # # # # 

In [3]:
print("Grid Map with Shadow:")
grid_map.print_grid(shadow=shadow)

Grid Map with Shadow:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # . . . . . . . . . . # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # . . . . . . . . . . # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # . . . . . . . . . . # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # . . . . . . . . . . . . . # # # # # # # # # # # #

In [4]:
# from abstractions.goap.spatial import GridMap, GameEntity, BlocksMovement, BlocksLight
# import random

# # Create a grid map
# grid_map = GridMap(width=20, height=20)

# # Place walls randomly
# for _ in range(50):
#     x = random.randint(0, grid_map.width - 1)
#     y = random.randint(0, grid_map.height - 1)
#     wall = GameEntity(name=f"Wall_{x}_{y}", blocks_movement=BlocksMovement(value=True), blocks_light=BlocksLight(value=True))
#     grid_map.get_node((x, y)).add_entity(wall)

# # Place the player entity
# player_pos = (15, 1)
# player = GameEntity(name="Player")
# grid_map.get_node(player_pos).add_entity(player)

# # Find the shortest path from the player's position to a random goal position
# goal_pos = (grid_map.width - 2, grid_map.height - 2)
# start_node = grid_map.get_node(player_pos)
# goal_node = grid_map.get_node(goal_pos)
# path = grid_map.a_star(start_node, goal_node)

# # Get the radius, shadow, and raycast
# radius = grid_map.get_radius(start_node, max_radius=5)
# shadow = grid_map.get_shadow(start_node, max_radius=10)
# raycast = grid_map.get_raycast(start_node, goal_node)

# # Print the grid map with path, radius, shadow, and raycast
# print("Grid Map:")
# grid_map.print_grid(path=path)
# print("\nShadow:")
# grid_map.print_grid(shadow=shadow)


ValidationError: 1 validation error for RayCast
nodes
  Value error, The raycast path should be empty if there is no clear path [type=value_error, input_value=[Node(name='', id='92a7de...at 0x000001FC3778CB80>)], input_type=list]
    For further information visit https://errors.pydantic.dev/2.1.2/v/value_error

In [ ]:
from abstractions.goap.spatial import GridMap, GameEntity, BlocksMovement, BlocksLight

# Create a grid map
grid_map = GridMap(width=20, height=20)

# Create walls for the room
for x in range(5, 15):
    for y in range(5, 15):
        if x == 5 or x == 14 or y == 5 or y == 14:
            wall = GameEntity(name=f"Wall_{x}_{y}", blocks_movement=BlocksMovement(value=True), blocks_light=BlocksLight(value=True))
            grid_map.get_node((x, y)).add_entity(wall)

# Create an opening in the room
opening_pos = (9, 5)
grid_map.get_node(opening_pos).entities.clear()

# Place the player entity inside the room
player_pos_inside = (9, 7)
player_inside = GameEntity(name="Player_Inside")
grid_map.get_node(player_pos_inside).add_entity(player_inside)

# Place the player entity at the boundary of the room
player_pos_boundary = (9, 5)
player_boundary = GameEntity(name="Player_Boundary")
grid_map.get_node(player_pos_boundary).add_entity(player_boundary)

# Place the player entity outside the room
player_pos_outside = (9, 3)
player_outside = GameEntity(name="Player_Outside")
grid_map.get_node(player_pos_outside).add_entity(player_outside)

# Test shadowcast inside the room
shadow_inside = grid_map.get_shadow(grid_map.get_node(player_pos_inside), max_radius=10)
print("Shadowcast inside the room:")
grid_map.print_grid(shadow=shadow_inside)

# Test shadowcast at the boundary of the room
shadow_boundary = grid_map.get_shadow(grid_map.get_node(player_pos_boundary), max_radius=10)
print("\nShadowcast at the boundary of the room:")
grid_map.print_grid(shadow=shadow_boundary)

# Test shadowcast outside the room
shadow_outside = grid_map.get_shadow(grid_map.get_node(player_pos_outside), max_radius=10)
print("\nShadowcast outside the room:")
grid_map.print_grid(shadow=shadow_outside)

# Test raycast inside the room
raycast_inside = grid_map.get_raycast(grid_map.get_node(player_pos_inside), grid_map.get_node((12, 12)))
print("\nRaycast inside the room:")
grid_map.print_grid(raycast=raycast_inside)

# Test raycast that should break (blocked by wall)
print("\nRaycast that should break (blocked by wall):")

try:
    raycast_break = grid_map.get_raycast(grid_map.get_node(player_pos_inside), grid_map.get_node((9, 19)))
    grid_map.print_grid(raycast=raycast_break)
except ValidationError as e:
    print(e)

Shadowcast inside the room:
. . . . . . . . + + + . . . . . . . . . 
. . . . . . . . + + + . . . . . . . . . 
. . . . . . . . + + + . . . . . . . . . 
. . . . . . . . . + . . . . . . . . . . 
. . . . . . . . . + . . . . . . . . . . 
. . . . . # # # # + # # # # # . . . . . 
. . . . . # . . . + . . . . # . . . . . 
. . . . . # . . . S . . . . # . . . . . 
. . . . . # . . . . . . . . # . . . . . 
. . . . . # . . . . . . . . # . . . . . 
. . . . . # . . . . . . . . # . . . . . 
. . . . . # . . . . . . . . # . . . . . 
. . . . . # . . . . . . . . # . . . . . 
. . . . . # . . . . . . . . # . . . . . 
. . . . . # # # # # # # # # # . . . . . 
. . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . 

Shadowcast at the boundary of the room:
. . . . + + + + + + + + + + + . . . . . 
. . . . . + + + + + + + + + . . . . . . 
. . . . . . + + + + + + + . .

ValidationError: 1 validation error for RayCast
nodes
  Value error, The raycast path should be empty if there is no clear path [type=value_error, input_value=[Node(name='', id='51dbc4...at 0x000001F591CD0EB0>)], input_type=list]
    For further information visit https://errors.pydantic.dev/2.1.2/v/value_error

In [ ]:
grid_map.get_raycast(grid_map.get_node(player_pos_inside), grid_map.get_node((19, 12)))

In [ ]:
grid_map.get_raycast(grid_map.get_node(player_pos_inside), grid_map.get_node((9, 15))).nodes